# 4_Model_Testing

En este notebook trataremos de poner a prueba nuestros modelos entrenados con nuevos datos. Desde que se genero el dataset hasta la fecha actual 07-Sep, Han salido nuevos hits que veremos que modelo de los anteriores es capaz de predecir mas hits.

In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '../Data_Generation')
from music_data import Music_Data
import joblib

In [2]:
df_20 = pd.read_csv("../Data/top_100_Billboard_2020.csv")

In [3]:
df_20['date_chart'].max()

'2020-08-08'

In [4]:
# sp_cid: Sustituir por spotify client id (str)
# sp_secret: Sustituir por spotify secret id (str)
# ge_token: Sustituir Genius secret id (str)
data_music = Music_Data("sp_cid", "sp_secret", "ge_token")

In [5]:
df_test = data_music.Get_Billboard_Data("09-08-2020","13-09-2020")

Added song: Watermelon Sugar Artist: Harry Styles
Added song: Rockstar Artist: DaBaby 
Added song: Whats Poppin Artist: Jack Harlow 
Added song: Blinding Lights Artist: The Weeknd
Added song: Roses Artist: SAINt JHN
Added song: My Future Artist: Billie Eilish
Added song: Savage Love (Laxed - Siren Beat) Artist: Jawsh 685 x Jason Derulo
Added song: Cardigan Artist: Taylor Swift
Added song: Go Crazy Artist: Chris Brown & Young Thug
Added song: Blueberry Faygo Artist: Lil Mosey
Added song: Before You Go Artist: Lewis Capaldi
Added song: Adore You Artist: Harry Styles
Added song: Savage Artist: Megan Thee Stallion 
Added song: Break My Heart Artist: Dua Lipa
Added song: I Hope Artist: Gabby Barrett
Added song: Circles Artist: Post Malone
Added song: Come & Go Artist: Juice WRLD x Marshmello
Added song: Popstar Artist: DJ Khaled 
Added song: Move Ya Hips Artist: A$AP Ferg 
Added song: For The Night Artist: Pop Smoke 
Added song: Wishing Well Artist: Juice WRLD
Added song: Don't Start Now Ar

In [6]:
df_test.to_csv("../Data/test_hits.csv", index = False, header=True)

In [7]:
df_20.drop(df_20.columns[3:30], axis = 1, inplace = True) 
df_test.drop(df_test.columns[3:30], axis = 1, inplace = True) 

In [8]:
df_uniq = df_20.merge(df_test, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

In [9]:
df_uniq = df_uniq.drop_duplicates(subset = ["id"])
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es la misma canción
# por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la canción, para solventar esta
# casuistica.
df_uniq = df_uniq.drop_duplicates(subset = ["artist","title"], keep="first")

In [10]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from dateutil.parser import parse

client_credentials_manager = SpotifyClientCredentials(client_id='01ec25b32db847e3aeb622ebe1a894c8', client_secret='cd8d3b7666164ea5943688215d9226c2') 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=50)

def find_song(id_track, year):
    try:
        if year == False:
            track = sp.track(id_track)
            track_feat = sp.audio_features(id_track)
            song = [[track_feat[0]["danceability"], track_feat[0]["energy"], track_feat[0]["key"],
            track_feat[0]["loudness"], track_feat[0]["mode"], track_feat[0]["speechiness"],
            track_feat[0]["acousticness"], track_feat[0]["instrumentalness"], track_feat[0]["liveness"],
            track_feat[0]["valence"], track_feat[0]["tempo"],track_feat[0]["time_signature"],
                    track["duration_ms"]]]
        else:
            track = sp.track(id_track)
            get_year = lambda date: parse(date, fuzzy=True).year
            year_realease = get_year(track["album"]["release_date"])
            track_feat = sp.audio_features(id_track)
            song = [[year_realease, track_feat[0]["danceability"], track_feat[0]["energy"],
                     track_feat[0]["key"],track_feat[0]["loudness"], track_feat[0]["mode"],
                     track_feat[0]["speechiness"],track_feat[0]["acousticness"],
                     track_feat[0]["instrumentalness"], track_feat[0]["liveness"],track_feat[0]["valence"],
                     track_feat[0]["tempo"],track_feat[0]["time_signature"],track["duration_ms"]]]

        return song
    except:
        print ("ERROR: Titulo no encontrado")

In [11]:
filename1 = '../Models/forest_model_random.pkl'
filename2 = '../Models/ada_boost_model_random.pkl'
filename3 = '../Models/light_gbm_model_random.pkl'
filename4 = '../Models/forest_model_random_year.pkl'
filename5 = '../Models/ada_boost_model_random_year.pkl'
filename6 = '../Models/light_gbm_model_random_year.pkl'
filename7 = '../Models/forest_model_random_93_20.pkl'
filename8 = '../Models/ada_boost_model_random_93_20.pkl'
filename9 = '../Models/light_gbm_model_random_93_20.pkl'
filename10 = '../Models/forest_model_random_year_93_20.pkl'
filename11 = '../Models/ada_model_random_year_93_20.pkl'
filename12 = '../Models/light_gbm_model_random_year_93_20.pkl'

In [12]:
forest_random = joblib.load(filename1) 
ada_random = joblib.load(filename2) 
lgb_random = joblib.load(filename3) 
forest_random_year = joblib.load(filename4) 
ada_random_year = joblib.load(filename5) 
lgb_random_year = joblib.load(filename6) 
forest_random_93 = joblib.load(filename7) 
ada_random_93 = joblib.load(filename8) 
lgb_random_93 = joblib.load(filename9) 
forest_random_year_93 = joblib.load(filename10) 
ada_random_year_93 = joblib.load(filename11) 
lgb_random_year_93 = joblib.load(filename12) 

In [15]:
df_uniq['proba_forest_random']= 0 
df_uniq['proba_ada_random']= 0 
df_uniq['proba_lgb_random']= 0 
df_uniq['proba_forest_random_year']= 0 
df_uniq['proba_ada_random_year']= 0 
df_uniq['proba_lgb_random_year']= 0 
df_uniq['proba_forest_random_93']= 0 
df_uniq['proba_ada_random_93']= 0 
df_uniq['proba_lgb_random_93']= 0 
df_uniq['proba_forest_random_year_93']= 0 
df_uniq['proba_ada_random_year_93']= 0 
df_uniq['proba_lgb_random_year_93']= 0 
for i, item in df_uniq.iterrows():
    song = find_song(item['id'],False)
    core = forest_random.predict_proba(song)
    df_uniq.loc[i,"proba_forest_random"]=score[0][1]
    score = ada_random.predict_proba(song)
    df_uniq.loc[i,"proba_ada_random"]=score[0][1]
    score = lgb_random.predict_proba(song)
    df_uniq.loc[i,"proba_lgb_random"]=score[0][1]
    
    song_year = find_song(item['id'],True)
    score = forest_random_year.predict_proba(song_year)
    df_uniq.loc[i,"proba_forest_random_year"]=score[0][1]
    score = ada_random_year.predict_proba(song_year)
    df_uniq.loc[i,"proba_ada_random_year"]=score[0][1]
    score = lgb_random_year.predict_proba(song_year)
    df_uniq.loc[i,"proba_lgb_random_year"]=score[0][1]
    
    score = forest_random_93.predict_proba(song)
    df_uniq.loc[i,"proba_forest_random_93"]=score[0][1]
    score = ada_random_93.predict_proba(song)
    df_uniq.loc[i,"proba_ada_random_93"]=score[0][1]
    score = lgb_random_93.predict_proba(song)
    df_uniq.loc[i,"proba_lgb_random_93"]=score[0][1]
    
    score = forest_random_year_93.predict_proba(song_year)
    df_uniq.loc[i,"proba_forest_random_year_93"]=score[0][1]
    score = ada_random_year_93.predict_proba(song_year)
    df_uniq.loc[i,"proba_ada_random_year_93"]=score[0][1]
    score = lgb_random_year_93.predict_proba(song_year)
    df_uniq.loc[i,"proba_lgb_random_year_93"]=score[0][1]

In [16]:
print('proba_forest_random', len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random']<0.5]))
print ('proba_ada_random',len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random']<0.5]))
print ('proba_lgb_random',len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random']<0.5]))
print('proba_forest_random_year',len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random_year']<0.5]))
print ('proba_ada_random_year',len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random_year']<0.5]))
print ('proba_lgb_random_year',len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_year']<0.5]))
print ('proba_forest_random_93',len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random_93']<0.5]))
print ('proba_ada_random_93',len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random_93']<0.5]))
print('proba_lgb_random_93',len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_93']<0.5]))
print('proba_forest_random_year_93',len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random_year_93']<0.5]))
print('proba_ada_random_year_93',len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random_year_93']<0.5]))
print('proba_lgb_random_year_93',len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_year_93']<0.5]))

proba_forest_random 10
proba_ada_random 25
proba_lgb_random 21
proba_forest_random_year 4
proba_ada_random_year 4
proba_lgb_random_year 5
proba_forest_random_93 34
proba_ada_random_93 32
proba_lgb_random_93 31
proba_forest_random_year_93 10
proba_ada_random_year_93 13
proba_lgb_random_year_93 10


In [17]:
len_df_uniq = len (df_uniq)
proba_forest_random = (len_df_uniq-len(df_uniq[df_uniq['proba_forest_random']<0.5]))*100/len_df_uniq
print('proba_forest_random: {:.2f}'.format(proba_forest_random))
proba_ada_random = (len_df_uniq-len(df_uniq[df_uniq['proba_ada_random']<0.5]))*100/len_df_uniq
print ('proba_ada_random: {:.2f}'.format(proba_ada_random))
proba_lgb_random = (len_df_uniq-len(df_uniq[df_uniq['proba_lgb_random']<0.5]))*100/len_df_uniq
print ('proba_lgb_random: {:.2f}'.format(proba_lgb_random))
proba_forest_random_year = (len_df_uniq-len(df_uniq[df_uniq['proba_forest_random_year']<0.5]))*100/len_df_uniq
print('proba_forest_random_year: {:.2f}'.format(proba_forest_random_year))
proba_ada_random_year = (len_df_uniq-len(df_uniq[df_uniq['proba_ada_random_year']<0.5]))*100/len_df_uniq
print ('proba_ada_random_year: {:.2f}'.format (proba_ada_random_year))
proba_lgb_random_year = (len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_year']<0.5]))*100/len_df_uniq
print ('proba_lgb_random_year: {:.2f}'.format (proba_lgb_random_year))
proba_forest_random_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random_93']<0.5]))*100/len_df_uniq
print ('proba_forest_random_93: {:.2f}'.format(proba_lgb_random_year))
proba_ada_random_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random_93']<0.5]))*100/len_df_uniq
print ('proba_ada_random_93: {:.2f}'.format(proba_ada_random_93))
proba_lgb_random_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_93']<0.5]))*100/len_df_uniq
print('proba_lgb_random_93: {:.2f}'.format(proba_lgb_random_93))
proba_forest_random_year_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_forest_random_year_93']<0.5]))*100/len_df_uniq
print('proba_forest_random_year_93: {:.2f}'.format(proba_forest_random_year_93))
proba_ada_random_year_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_ada_random_year_93']<0.5]))*100/len_df_uniq
print('proba_ada_random_year_93: {:.2f}'.format(proba_ada_random_year_93))
proba_lgb_random_year_93 = (len (df_uniq)-len(df_uniq[df_uniq['proba_lgb_random_year_93']<0.5]))*100/len_df_uniq
print('proba_lgb_random_year_93: {:.2f}'.format(proba_lgb_random_year_93))

proba_forest_random: 23.81
proba_ada_random: 59.52
proba_lgb_random: 50.00
proba_forest_random_year: 9.52
proba_ada_random_year: 9.52
proba_lgb_random_year: 11.90
proba_forest_random_93: 11.90
proba_ada_random_93: 76.19
proba_lgb_random_93: 73.81
proba_forest_random_year_93: 23.81
proba_ada_random_year_93: 30.95
proba_lgb_random_year_93: 23.81
